In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
from collections import defaultdict
import os
import re

# import chromadb
from llama_index.core import Document, VectorStoreIndex, set_global_handler
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
    MarkdownNodeParser,
)
from llama_index.core.vector_stores.types import VectorStoreQueryMode
# from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.vector_stores.milvus import MilvusVectorStore
# from llama_index.vector_stores.qdrant import QdrantVectorStore
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
# from llama_index.embeddings.voyageai import VoyageEmbedding
import nest_asyncio
import optuna
import pandas as pd
# from pymilvus import MilvusClient
# from qdrant_client import QdrantClient, AsyncQdrantClient

from utils.retrieve import (
    generate_ngrams_from_text,
    generate_ngrams_from_texts,
    f_score
)

In [ ]:
# configure
# filename = 'everdell.md'
qa_filename = 'question_list.csv'
ngram_size = 2  # use 2 instead of 3 so we don't skip 2-word header chunks
f_beta = 3      # weight recall 3 times as important as precision in f-score
n_trials = 25   # number of Optuna trials

pd.set_option('display.max_colwidth', None)

In [ ]:
# qa_df = pd.read_csv(f'data/{qa_filename}', na_filter=False)
qa_df = pd.read_csv(qa_filename, na_filter=False)
print(len(qa_df))
qa_df.head(3)

In [ ]:
# NOTE: we shouldn't include questions in the *test* set right now,
# but people are still adding the manual quotes, 
# and since we have so few questions with manual quotes so far 
# we will use all of them for this demo.

# keep only rows with at least 1 manual quote
qa_df = qa_df[qa_df['Quotes'].notna() & (qa_df['Quotes'] != '')]
print(len(qa_df))

In [ ]:
from utils.retrieve import extract_question_ngrams

# generate bigrams (ngram size=2) for each manual quote
# and store them in the question_ngrams dictionary
question_ngrams = extract_question_ngrams(qa_df, ngram_size)